# Lab 1 - komunikacja HTTP

## Wysyłanie żądań

W celu wysyłania żądań do serwera należy wykorzystać dedykowaną bibliotekę. W języku Python najpopularniejsza jest biblioteka [requests](https://pypi.org/project/requests/), która nie wchodzi w skład biblioteki standardowej, zatem należy ją zainstalować.

Każde żądanie zawiera parametry takie jak:
- adres URL (endpoint),
- metodę (GET, POST, PUT, PATCH, DELETE, OPTIONS, ...),
- nagłówek,
- body (dostępne w niektórych metodach protokołu HTTP).

Odpowiedź zwracana przez serwer różni się od żądania istnieniem dodatkowego parametru w postaci numerycznego kodu odpowiedzi, który informuje o statusie operacji. Kody odpowiedzi można podzielić na następujące grupy:
- 100-199 - informacyjne,
- 200-299 - pomyślne,
- 300-399 - przekierowujące,
- 400-499 - błąd klienta (nieprawidłowe żądanie),
- 500-599 - błąd przetwarzania żądania po stronie serwera.

In [ ]:
import requests

url = 'http://wmii.uwm.edu.pl/'
response = requests.get(url)

print('Kod odpowiedzi ', response.status_code)
print('Tresc odpowiedzi ', response.content)
print('Naglowki ', response.headers)

## Interpretacja odpowiedzi na żądania

Serwery, w zależności od oprogramowania, mogą zwracać odpowiedzi w różnych formatach. Najpopularniejszym jest HTML, który zawiera definicję wizualną aplikacji. W celu ekstrakcji wybranych informacji należy dokonać ekstrakcji treści zakodowanych w drzewie DOM za pomocą języka HTML. Najpopularniejszą metodą jest wykorzystanie biblioteki [beautifulsoup4](https://pypi.org/project/beautifulsoup4/), która podobnie jak requests, nie wchodzi w skład biblioteki standardowej.

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'http://wmii.uwm.edu.pl/kadra'
response = requests.get(url)

soup = BeautifulSoup(response.content)
table = soup.find('table', class_='views-table cols-8').find('tbody')

for row in table:
  degree = row.find('td', class_='views-field views-field-degree').text.strip()
  name = row.find('td', class_='views-field views-field-title active').text.strip()
  phone = row.find('td', class_='views-field views-field-field-phone').text.strip()

  print(degree, name, phone)

## Komunikcja z REST API

Obecnie coraz więcej aplikacji internetowych jest transformowanych do postaci REST API, co oznacza że po stronie serwera znajduje się interfejs programistyczny umożliwiający sprawną komunikację za pomocą formatu JSON. Rozwiązanie takie ma wiele zalet. Najważniejszą jest możliwość umożliwienia komunikacji nie tylko za pomocą przeglądarki internetowej, ale także innym kompletnym aplikacjom (np. mobilnym).

W poniższym przykładzie wykorzystamy otwarte REST API dostarczające informacji o jakości powietrza w Polsce. Dokumentację interfejsu można znaleźć pod adresem: [https://powietrze.gios.gov.pl/pjp/content/api](https://powietrze.gios.gov.pl/pjp/content/api).

W celu uzyskania listy stacji pomiarowych należy wykonać żądanie metodą GET pod adres: [https://api.gios.gov.pl/pjp-api/rest/station/findAll](https://api.gios.gov.pl/pjp-api/rest/station/findAll)

In [ ]:
import requests
import json

url = 'https://api.gios.gov.pl/pjp-api/rest/station/findAll'
response = requests.get(url)

response.content

Warto zauważyć, że odpowiedź przychodząca z serwera zawiera "nietypowe" znaki, co wskazuje na kodowanie w stylu Unix'a. Z łatwością można zmodyfikować kodowanie uzyskanych danych (jak zaprezentowano poniżej) do najpopularniejszego formatu UTF-8, który umożliwia czytelną reprezentację alfabetu łacińskiego.

In [ ]:
response.content.decode('utf-8')

Odpowiedź z serwera jest reprezentowana za pomocą formatu JSON, co oznacza że jest słownikiem lub listą słowników, gdzie każdy obiekt jest reprezentowany za pomocą niezależnego słownika. Można więc z łatwością tekst zwrócony przez serwer zamienić do postaci obiektu słownikowego w języku Python. Służy do tego biblioteka json włączona w skład biblioteki standardowej.

In [ ]:
content = response.content.decode('utf-8')
parsed_content = json.loads(content)

print(type(response.content), type(content), type(parsed_content))

In [ ]:
parsed_content

Posiadając odpowiedź z serwera w postaci listy słowników języka Python można z łatwością dokonać sortowania, filtrowania itd. uzyskanych informacji.

In [ ]:
for station in parsed_content:
  print(f'ID: {station["id"]}, nazwa: {station["stationName"]}, miasto: {station["city"]["name"]}, lokalizacja: {station["addressStreet"]}')

Mając odseparowane informacje o identyfikatorach stacji pomiarowych można pójśc o krok dalej i za pomocą adresu [https://api.gios.gov.pl/pjp-api/rest/station/sensors/nnn](https://api.gios.gov.pl/pjp-api/rest/station/sensors/nnn) uzyskać listę czujników w stacji pomiarowej o identyfikatorze **nnn**.

In [ ]:
import requests

station_id = 877
url = f'https://api.gios.gov.pl/pjp-api/rest/station/sensors/{station_id}'
response = requests.get(url)

if response.status_code != 200:
  exit()

stations = json.loads(response.content.decode('utf-8'))

stations

W ostatnim kroku dla uzyskanego identyfikatora czujnika można uzyskać wyniki pomiarów z ostatnich godzin wywołując żądanie pod adresem [https://api.gios.gov.pl/pjp-api/rest/data/getData/mmm](https://api.gios.gov.pl/pjp-api/rest/data/getData/mmm), gdzie **mmm** oznacza identyfikator czujnika.

In [ ]:
import requests
import json

sensor_id = 5766
url = f'https://api.gios.gov.pl/pjp-api/rest/data/getData/{sensor_id}'
response = requests.get(url)

if response.status_code != 200:
  assert False

data = json.loads(response.content.decode('utf-8'))
value = data['values'][0]

print(f'Czas: {value["date"]}, wartosc odczytu: {value["value"]}')

## Zadania

1. Przygotować funkcję check_url(url: str) -> bool, która przyjmie adres URL i zwróci wartość logiczną informującą o tym czy serwer zwrócił odpowiedź poprawną. Za odpowiedź poprawną należy uznać taką, której kod mieści się w zakresie 200-299.

2. Przygotować kod, który dla wskazanej nazwy miejscowości wygeneruje liniowy wykres temperatury dla najbliższych godzin. Jako źródło danych można wykorzystać adres: https://www.meteoprog.pl/pl/weather/NazwaMiasta/, gdzie frazę **NazwaMiasta** należy zastąpić właściwą nazwą miasta/miejscowości. Przy implementacji rozwiązania proszę wykorzystać bibliotekę BeautifulSoup. Do wizualizacji można wykorzystać dowolne narzędzie.

3. Przygotować kod, który wygeneruje wieloserjny wykres liniowy wartości odczytów wybranych czujników wybranej stacji pomiarowej z ostatnich godzin. Jako źródło danych  wykorzystać API GIOŚ.

4. Która z automatycznych metod ekstrakcji informacji z aplikacji internetowych jest wygodniejsza?